# Домашнее задание 3

## Сбор данных из интернета и генерация текстов

В этом домашнем задании вам предстоит: 
1. выбрать свой источник структурированных данных (прогноз погоды, курс акций, счет спортивных мероприятий, или что-нибудь еще) 
2. научиться извлекать данные из с выбранной веб-страницы
3. научиться генерировать текст по шаблонам

## Часть 1

Выберете источник данных, который вам симпатичен. Извлеките из html-кода страницы температуру сегодня, уровень осадков и скорость ветра.

In [147]:
import requests
from scrapy.selector import Selector
from scrapy.http import HtmlResponse, Request

In [148]:
def load_html(url):
    html = requests.get(url).text
    return HtmlResponse(url, body=html, encoding='utf-8')

In [66]:
class Wind(object):
    def __init__(self, speed=None, speed_unit=None, direction=None):
        self.speed = speed
        self.speed_unit = speed_unit
        self.direction = direction
        
    def __str__(self):
        return "{:.1f} {} {}".format(self.speed, self.speed_unit, self.direction)        

class Weather(object):
    def __init__(self, temp=None, wind=None, humidity=None):
        self.temp = temp
        self.wind = wind
        self.humidity = humidity
        
    def __str__(self):
        return "{:+d}, влажность {}%, ветер {}".format(self.temp, self.humidity, self.wind)
    
    def __repr__(self):
        return self.__str__()

In [98]:
def get_now_weather(city='moscow'):
    url = 'https://yandex.ru/pogoda/{}/'.format(city)
    html = load_html(url)
    temp = html.css('.fact .fact__temp .temp__value::text').extract_first()
    humidity = html.css('.fact .fact__humidity .term__value::text').extract_first()
    wind = html.css('.fact .fact__wind-speed .term__value ::text').extract()
    wind_speed, _, wind_speed_unit, wind_direction = wind
    return Weather(
        temp=int(temp),
        wind=Wind(
            float(wind_speed.replace(',', '.')), 
            wind_speed_unit.strip(' ,'), 
            wind_direction
        ),
        humidity=int(humidity.strip('%'))
    )

In [99]:
get_now_weather('moscow')

+1, влажность 92%, ветер 2.0 м/с Ю

## Часть 2
Запрограммируйте шаблон, описывающие температуру, уровень осадков и скорость ветра сегодня. Например,
"Температура сегодня составляет X градусов", где Х – излеченное выше число. Сгенерируйте таким образом описание погоды сегодня. Учтите, величина X влияет на грамматическое число, использованное в единицах измерения ("двадцать градусов", но "двадцать один градус"). Используйте бибилиотеку utg или дополнительные функции pymorphy2

In [138]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
degrees = morph.parse('градус')[0]

def tpl_temp(city, temp):
    city = morph.parse(city)[0]    
    return "Температура в {city} сейчас {temp} {unit}".format(
        city=city.inflect({'loct'}).word.title(),
        temp=temp,
        unit=degrees.make_agree_with_number(temp).word
    )

In [139]:
weather_moscow = get_now_weather('moscow')
print(tpl_temp('Москва', weather_moscow.temp))

Температура в Москве сейчас 1 градус


In [140]:
weather_amsterdam = get_now_weather('10466')
print(tpl_temp('Амстердам', weather_amsterdam.temp))

Температура в Амстердаме сейчас 10 градусов


## Часть 3 (необязательная)
Соберите данные не только за сегодняшний, но и за вчерашний день, и напишите шаблоны, которые сравнивают вчерашние и сегодняшние показатели (например, "сегодня похолодало на Х градусов по сравнению со вчера").

In [145]:
def tpl_compare_temp(city1, temp1, city2, temp2):
    if temp2 == temp1:
        compare_text = "так же, как"
        
    else:
        diff=abs(temp2 - temp1)
        compare_text = "на {diff} {unit} {less_more} чем".format(
            diff=diff,
            unit=degrees.make_agree_with_number(diff).word,
            less_more='холоднее' if temp1 < temp2 else 'теплее'
        )
    
    return "{city1_temp} ({compare_text} в {city2})".format(
        city1_temp=tpl_temp(city1, temp1),
        compare_text=compare_text,
        city2=morph.parse(city2)[0].inflect({'loct'}).word.title()
    )    

In [146]:
print(tpl_compare_temp('Москва', weather_moscow.temp, 'Амстердам', weather_amsterdam.temp))

Температура в Москве сейчас 1 градус (на 9 градусов холоднее чем в Амстердаме)
